In [ ]:
import torchio as tio
from pathlib import Path
import matplotlib.animation as animation
import matplotlib.pyplot as plt

img = Path('/storage_bizon/naravich/3D_Shockwave/Image/')
cal = Path('/storage_bizon/naravich/3D_Shockwave/Calcium/')
img = sorted(list(img.glob('*.nii.gz')))
cal = sorted(list(cal.glob('*.nii.gz')))
subject_a = tio.Subject(
    image=tio.ScalarImage(img[3]),
    label=tio.LabelMap(cal[3]),
    diagnosis='positive',
)
subject_a.plot()

In [ ]:
import numpy as np
img = np.load('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_preprocessed/Dataset301_Calcium_OCT/nnUNetPlans_3d_fullres/101-019.npy')
img = np.transpose(img, (0, 2, 3, 1))
# img = img[np.newaxis,]
img.shape

In [ ]:
import torchio as tio

subject_a = tio.Subject(
    image=tio.ScalarImage(tensor=img),
    genesis=tio.ScalarImage('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/GenesisTrainer__nnUNetPlans__3d_fullres/fold_1/validation/101-019.nii.gz'),
    nnUnet=tio.ScalarImage('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/validation/101-019.nii.gz'),
    gt=tio.ScalarImage('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_preprocessed/Dataset301_Calcium_OCT/gt_segmentations/101-019.nii.gz')

)
subject_a.plot()

In [ ]:
image = subject_a.image.numpy()
label = subject_a.label.numpy()

In [ ]:
import numpy as np
np.unique(label)

In [ ]:
positives = []
for i in range(label.shape[3]):
    if label[0, :, :, i].sum() > 0:
        positives.append(i) 
print(positives)

In [ ]:
image = image[:, :, :, positives]
label = label[:, :, :, positives]


In [ ]:
fig, ax = plt.subplots(1, 2)
im0 = ax[0].imshow(image[0][:, :, 0], animated=True, cmap='gray')
im1 = ax[1].imshow(label[0][:, :, 0], animated=True, cmap='gray')

def update(i):
    im0.set_array(label[0][:, :, i])
    im1.set_array(image[0][:, :, i])
    return im1, 

# Create the animation object
animation_fig = animation.FuncAnimation(fig, update, frames=label[0].shape[2], interval=1, blit=True,repeat_delay=10,)

# Show the animation
plt.show()

animation_fig.save("step.gif")